# Set up environment

In [4]:
import os

from azure.identity import DefaultAzureCredential
from azure.mgmt.core.tools import parse_resource_id
from dotenv import load_dotenv

credentials = DefaultAzureCredential()

In [61]:
load_dotenv(override=True)

True

In [62]:
FOUNDRY_AOAI_ENDPOINT = os.getenv("FOUNDRY_AOAI_ENDPOINT")
FOUNDRY_AOAI_KEY = os.getenv("FOUNDRY_AOAI_KEY")
AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
AI_SEARCH_INDEX_NAME = os.getenv("AI_SEARCH_INDEX_NAME")
KNOWLEGE_BASE_NAME = os.getenv("KNOWLEGE_BASE_NAME")
KNOWLEGE_SOURCE_NAME = os.getenv("KNOWLEGE_SOURCE_NAME")
KNOWLEGE_BASE_MODEL = os.getenv("KNOWLEGE_BASE_MODEL")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")

# Set up Azure AI Search

## Create Index

In [5]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
  AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, HnswAlgorithmConfiguration,
  SearchField, SearchIndex, SemanticConfiguration, SemanticField, SemanticPrioritizedFields,
  SemanticSearch, VectorSearch, VectorSearchProfile
)

In [56]:
search_fields = [
  SearchField(
    name="id", 
    type="Edm.String", 
    key=True, 
    filterable=True, 
    sortable=True, 
    facetable=True
  ),
  SearchField(
    name="chunk",
    type="Edm.String",
    filterable=False,
    sortable=True,
    searchable=True
  ),
  SearchField(
    name="contentVector",
    type="Collection(Edm.Single)",
    searchable=True,
    vector_search_dimensions=3072,
    vector_search_profile_name="hnswVectorConfig"
  ),
  SearchField(
    name="pageNumber",
    type="Edm.Int32",
    filterable=True,
    sortable=True,
    facetable=True
  )
]

In [64]:
vector_search_algorithm_config = HnswAlgorithmConfiguration(
  name="hnswVectorConfig",
  parameters={
    "m": 4,
    "efConstruction": 400,
    "efSearch": 100
  }
)

vector_search_vectorizer = AzureOpenAIVectorizer(
  vectorizer_name="aoaiVectorizerText3Large",
  parameters=AzureOpenAIVectorizerParameters(
    resource_url=FOUNDRY_AOAI_ENDPOINT,
    deployment_name=EMBEDDING_MODEL,
    model_name=EMBEDDING_MODEL,
    api_key=FOUNDRY_AOAI_KEY
  )
)

semantic_search_config = SemanticConfiguration(
  name="semantic_config",
  prioritized_fields=SemanticPrioritizedFields(
    content_fields=[SemanticField(field_name="chunk")]
  )
)

In [65]:
index = SearchIndex(
  name=AI_SEARCH_INDEX_NAME,
  fields=search_fields,
  vector_search=VectorSearch(
    profiles=[
      VectorSearchProfile(
        name="hnswVectorConfig",
        algorithm_configuration_name=vector_search_algorithm_config.name,
        vectorizer_name=vector_search_vectorizer.vectorizer_name
      )
    ],
    algorithms=[vector_search_algorithm_config],
    vectorizers=[vector_search_vectorizer]
  ),
  semantic_search=SemanticSearch(
    default_configuration_name=semantic_search_config.name,
    configurations=[semantic_search_config]
  )
)

In [79]:
index_client = SearchIndexClient(
  endpoint=AI_SEARCH_ENDPOINT,
  credential=credentials
)

index_client.create_or_update_index(index)
print(f"Index '{index.name}' created successfully.")

Index 'foundry-iq-index-1226' created successfully.


## Create Knowledge source

In [83]:
from azure.search.documents.indexes.models import (
  SearchIndexFieldReference, SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters
)

knowledge_source = SearchIndexKnowledgeSource(
  name=KNOWLEGE_SOURCE_NAME,
  description="Knowledge source for Foundry IQ Agent",
  search_index_parameters=SearchIndexKnowledgeSourceParameters(
    search_index_name=AI_SEARCH_INDEX_NAME,
    source_data_fields=[
      SearchIndexFieldReference(name="id"),
      SearchIndexFieldReference(name="pageNumber")
    ]
  )
)

index_client.create_or_update_knowledge_source(knowledge_source=knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created successfully.")


Knowledge source 'iq-knowledge-source-1226' created successfully.


## Create Knowledge Base

In [89]:
from azure.search.documents.indexes.models import (
  KnowledgeBase, KnowledgeRetrievalMinimalReasoningEffort,
  KnowledgeRetrievalOutputMode, KnowledgeSourceReference, KnowledgeBaseAzureOpenAIModel
)

knowledge_base = KnowledgeBase(
  name=KNOWLEGE_BASE_NAME,
  description="Knowledge base for Foundry IQ Agent",
  knowledge_sources=[
    KnowledgeSourceReference(
      name=KNOWLEGE_SOURCE_NAME
    )
  ],
  models=[
    KnowledgeBaseAzureOpenAIModel(
      azure_open_ai_parameters = AzureOpenAIVectorizerParameters(
        resource_url=f"{FOUNDRY_AOAI_ENDPOINT}/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview",
        deployment_name=KNOWLEGE_BASE_MODEL,
        model_name=KNOWLEGE_BASE_MODEL,
        api_key=FOUNDRY_AOAI_KEY
      )
    )
  ],
  output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
  retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort()
)

index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)
print(f"Knowledge base '{knowledge_base.name}' created successfully.")

Knowledge base 'iq-knowledge-base-1226' created successfully.


In [85]:
MCP_ENDPOINT = f"{AI_SEARCH_ENDPOINT}/knowledgebases/{KNOWLEGE_BASE_NAME}/mcp?api-version=2025-11-01-Preview"